In [1]:
import gymnasium as gym
from stable_baselines3 import PPO
import time

import imageio
from IPython.display import Video, display # per visualizzarlo
import os

In [2]:
# Ipreparametri dell'envrionment
hp_reset_noise_scale= 0.06976748570068636 # scala del rumore quando l'ambiente viene resettato 
hp_forward_reward_weight = 1.7031840270142826 # peso del reward per il movimento in avanti
hp_ctrl_cost_weight = 1.3736559851030032 # peso del reward per il controllo
hp_healthy_reward = 2.370527088282008 # reward per la salute

hp_contact_cost_weight = 5.099569789843523e-05
healthy_z = (0.27002630801618377, 1.2006270382609852)
contact_force = (-1.1373782536958372, 1.9183153315908629)

In [3]:
env_render = gym.make(
    "Ant-v5",
    reset_noise_scale=hp_reset_noise_scale,
    forward_reward_weight=hp_forward_reward_weight,
    ctrl_cost_weight=hp_ctrl_cost_weight,
    healthy_reward=hp_healthy_reward,
    contact_cost_weight=hp_contact_cost_weight,
    healthy_z_range=healthy_z,
    contact_force_range=contact_force,
    render_mode='rgb_array'
)
"""
Questo snippet di codice inizializza un ambiente Gym per il compito "Ant-v5" con specifici iperparametri.

Parametri:
- reset_noise_scale (float): Scala del rumore aggiunto allo stato iniziale durante il reset dell'ambiente.
- forward_reward_weight (float): Peso della ricompensa per il movimento in avanti.
- ctrl_cost_weight (float): Peso del costo associato alle azioni di controllo.
- healthy_reward (float): Ricompensa per mantenere uno stato di salute.
- contact_cost_weight (float): Peso del costo associato alle forze di contatto.
- healthy_z_range (tuple): Intervallo di valori z considerati sani per l'agente.
- contact_force_range (tuple): Intervallo di forze di contatto considerate accettabili.
- render_mode (str): Modalità di rendering dell'ambiente, impostata su 'human' per il rendering visivo.

Questa configurazione è tipicamente utilizzata negli esperimenti di apprendimento per rinforzo per addestrare e valutare agenti nell'ambiente "Ant-v5".
"""

'\nQuesto snippet di codice inizializza un ambiente Gym per il compito "Ant-v5" con specifici iperparametri.\n\nParametri:\n- reset_noise_scale (float): Scala del rumore aggiunto allo stato iniziale durante il reset dell\'ambiente.\n- forward_reward_weight (float): Peso della ricompensa per il movimento in avanti.\n- ctrl_cost_weight (float): Peso del costo associato alle azioni di controllo.\n- healthy_reward (float): Ricompensa per mantenere uno stato di salute.\n- contact_cost_weight (float): Peso del costo associato alle forze di contatto.\n- healthy_z_range (tuple): Intervallo di valori z considerati sani per l\'agente.\n- contact_force_range (tuple): Intervallo di forze di contatto considerate accettabili.\n- render_mode (str): Modalità di rendering dell\'ambiente, impostata su \'human\' per il rendering visivo.\n\nQuesta configurazione è tipicamente utilizzata negli esperimenti di apprendimento per rinforzo per addestrare e valutare agenti nell\'ambiente "Ant-v5".\n'

In [4]:
env_render.training = False # Setta l'environment in modalità di valutazione
env_render.norm_reward = False # Disabilita la normalizzazione della reward. Questo è importante per valutare correttamente il modello.

In [5]:
# Carica il modello salvato
model = PPO.load("ppo_Ant_model_PPO15")

# Reset dell'ambiente e inizializzazione delle variabili
obs, _ = env_render.reset()
done = False
frames = []

# Imposta il timer (durata in secondi)
max_duration = 25 
start_time = time.time()

# Loop di esecuzione fino a quando l'episodio non è terminato o il timer scade
while not done:
    # Predice l'azione basata sull'osservazione corrente
    action, _ = model.predict(obs)
    
    # Esegue l'azione nell'ambiente e ottiene la nuova osservazione e lo stato di completamento
    obs, _, done, _, _ = env_render.step(action)
    
    # Renderizza l'ambiente e salva il frame
    frame = env_render.render()
    frames.append(frame)
    
    # Pausa per rendere il rendering visibile
    time.sleep(0.01)
    
    # Controlla se il tempo massimo è stato raggiunto
    if time.time() - start_time >= max_duration:
        print("Timer raggiunto, interrompo il rendering.")
        break    

Timer raggiunto, interrompo il rendering.


In [6]:
# Specifica la cartella di destinazione
output_dir = '../../myLib/Ant_media'
os.makedirs(output_dir, exist_ok=True)
output_video_path = os.path.join(output_dir, 'rendering_ant_ppo15.mp4')
output_gif_path = os.path.join(output_dir, 'rendering_ant_ppo15.gif')

# Converti i frame in ndarray
frames_ndarray = [frame for frame in frames if frame is not None]

# Verifica che ci siano frame validi
if not frames_ndarray:
    raise ValueError("Nessun frame valido catturato durante il rendering.")

# Salva i frame come video usando imageio (assicurati di avere ffmpeg installato)
with imageio.get_writer(output_video_path, fps=30) as video_writer:
    for frame in frames_ndarray:
        video_writer.append_data(frame)

# Salva i frame come GIF
imageio.mimsave(output_gif_path, frames_ndarray, fps=30)

print("Rendering video e GIF completato.")

Rendering video e GIF completato.
